In [1]:
# load data from csv
# path for input and target data tables

diagnosis_path = '../discharge_me/data/test_phase_2/diagnosis_hadm.csv'
discharge_path ='../discharge_me/data/test_phase_2/discharge.csv'
edstays_path = '../discharge_me/data/test_phase_2/edstays.csv'
radiology_path = '../discharge_me/data/test_phase_2/radiology.csv'
triage_path = '../discharge_me/data/test_phase_2/triage.csv'
target_path = '../discharge_me/data/test_phase_2/discharge_target.csv'
discharge_sections_path = '../discharge_me/data/test_phase_2/discharge_sections.csv'
radiology_sections_path = '../discharge_me/data/test_phase_2/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

In [26]:
discharge_sections_df.head(4)

,Unnamed: 0,subject_id,hadm_id,CC,Service,Major Surgical Procedure,HPI,PMH,SOC,FH,...,Problem List,Physical Exam,Medication Lists,Pertinent Results,BHC,Transitional Issues,Disposition,Discharge Instructions,Followup Instructions,Discharge Diagnosis
0,0,10001884,24962904,Shortness of Breath,"MEDICINE\n \nAllergies: \nIV Dye, Iodine Conta...",N/A,Ms. ___ is a ___ female with history of \nCOPD...,- COPD/Asthma on home 2L O2\n- Atypical Chest ...,___,Mother with asthma and hypertension. Father wi...,...,,ADMISSION PHYSICAL EXAM:\n====================...,is accurate and complete.\n1. Acetaminophen 32...,ADMISSION LABS: \n=========================\n_...,Ms. ___ is a ___ female with history of \nCOPD...,==========================\n[] For pt's contin...,Extended Care\n \nFacility:\n___\n \n___ Diagn...,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",,PRIMARY:\nCOPD Exacerbation\n\nSECONDARY:\nAfi...
1,1,10003019,22774359,fever,MEDICINE\n \nAllergies: \nRagweed / morphine /...,none,Mr ___ is a ___ with h/o stage IV Hodgkins c1d...,"1. Sarcoidosis, dx skin bx: intestinal & pulmo...",___,"Mother: ___, cardiac disease. \nFather: diver...",...,,ADMISSION EXAM\nVitals: 124/67 on neosynephrin...,The Preadmission Medication list is accurate a...,ADMISSION LABS\n___ 10:40AM BLOOD WBC-0.2* RBC...,___ male with h/o Hodgkin's lymphoma C1D17 ABV...,,Home With Service\n \nFacility:\n___\n \nDisch...,"Dear Mr. ___,\n\nIt has been our pleasure to b...",,"Primary Diagnosis\nNeutropenic Fever, no sourc..."
2,2,10003299,29323205,"left leg weakness, falls",NEUROLOGY\n \nAllergies: \nIodine-Iodine Conta...,None,___ is a ___ RH female with a PMHx of\nparamed...,- prior paramedian pontine infarct (___) \n- r...,___,Mother had stroke in her ___ or ___. Her pate...,...,,Admission Exam:\nVitals: T: 97.4 P: 65 R: 16 ...,The Preadmission Medication list is accurate a...,___ 01:10PM GLUCOSE-125* UREA N-9 CREAT-0.9 ...,___ RH female with a PMHx of paramedian pontin...,,Home With Service\n \nFacility:\n___\n \nDisch...,Dear ___ were hospitalized due to symptoms of ...,,Ischemic stroke
3,3,10003502,20459702,Nausea,MEDICINE\n \nAllergies: \nnifedipine / Amitrip...,None,Ms. ___ is an ___ with atrial fibrillation/flu...,Hypertension/hyperlipidemia\nCoronary artery d...,___,Mother deceased at ___ yo from breast cancer. ...,...,,On admission: \nVS 97.4 140/P 62 18 96\nGEN Al...,The Preadmission Medication list is accurate a...,___ 10:15AM BLOOD WBC-6.4 RBC-3.64* Hgb-11.1* ...,Hospitalization Summary: \nMs. ___ is an ___ y...,- monitoring of volume status and titration of...,Home With Service\n \nFacility:\n___\n \nDisch...,It was a pleasure caring for you at ___ \n___....,,Primary: \nAcute diastolic CHF exacerbation\nN...


# Load model

In [2]:
# https://huggingface.co/NouRed/BioMed-Tuned-Llama-3-8b
# Installations
!pip install peft --quiet
!pip install bitsandbytes --quiet
!pip install transformers --quiet
!pip install flash-attn --no-build-isolation --quiet

# Imports
import torch
from peft import LoraConfig, PeftModel
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoModelForCausalLM)

# generate_prompt function
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""

# Model Loading Configuration
based_model_path = "meta-llama/Meta-Llama-3-8B"
lora_weights = "NouRed/BioMed-Tuned-Llama-3-8b"

load_in_4bit=True
bnb_4bit_use_double_quant=True
bnb_4bit_quant_type="nf4"
bnb_4bit_compute_dtype=torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    based_model_path,
    )

tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

# Load Base Model in 4 Bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype
)

base_model = AutoModelForCausalLM.from_pretrained(
    based_model_path,
    device_map="auto",
    attn_implementation="flash_attention_2", # I have an A100 GPU with 40GB of RAM 😎
    quantization_config=quantization_config,
    )

# Load Peft Model
peft_model = PeftModel.from_pretrained(
    base_model,
    lora_weights,
    torch_dtype=torch.float16,
    )



/home/ubuntu/discharge_me/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:

def generate_with_model(instruction, context_input, model):
    prompt = generate_prompt(instruction, context_input)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate Text
    with torch.no_grad():
      generation_output = model.generate(
        **inputs,
        max_new_tokens=min(2048, len(inputs)+512)
      )
    
    # Decode Output
    output = tokenizer.decode(
        generation_output[0], 
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True)
    
    print(output)
    response = output.split("### Response:")[1].strip()
    return response

# extract relevant information

In [11]:
# extract problem list
hadm_id = 22774359
# model = peft_model
model = base_model

topic = '''
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
from a patient's hospital admission. 
You will be provided with text and data from a specific patient's medical records and clinical notes
delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
Answer clearly and concisely.
'''.replace('\n', '')

prompt = '''
What were the acute and chronic medical conditions relevant to the patient's hospital admission?
The conditions should also include major diagnosis. The acute conditions relate to the major conditions or 
symptoms the patient presented with and were treated for initially.
The chronic conditions are related to their past medical history.
Do not include any conditions or symptoms that are denied by the patient.
Create a list with the headings "Acute Problems:" and "Chronic Problems:"
'''.replace('\n', '')

context_str =''
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
cols = ['CC']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = f"The chief complaint was - {context_data} \n"
cols = ['HPI']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The History of present illness: \n{context_data} \n"
# cols = ['Medication Lists']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
# cols = ['Pertinent Results']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
cols = ['Discharge Diagnosis']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"

df = diagnosis_df
cols = ['icd_title']
cur_df = df[df['hadm_id'] == hadm_id]
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "


instruction = f"Instruction:{topic} \n Prompt: {prompt}"
context_input = f"<text> Text:{context_str}</text> "

response = generate_with_model(instruction, context_input, model)
problem_list = response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
 Instruction: What were the acute and chronic medical conditions relevant to the patient's hospital admission?The conditions should also include major diagnosis. The acute conditions relate to the major conditions or symptoms the patient presented with and were treated for initially.The chronic conditions are related to their past medical history.Do not include any conditions or symptoms that are denied by the patient.Create a list

In [35]:
# extract HPI summary USE THIS
hadm_id = 22774359
# model = peft_model
model = base_model

topic = '''
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
from a patient's hospital admission. 
You will be provided with text and data from a specific patient's medical records and clinical notes
delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
Answer clearly and concisely.
'''.replace('\n', '')

prompt = '''
Using the following patient specific data,
briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
Then, create a list of medical conditions and the associated treatments.
'''.replace('\n', '')

context_str =''
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
cols = ['CC']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = f"The chief complaint was - {context_data} \n"
cols = ['HPI']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The History of present illness: \n{context_data} \n"
# cols = ['Medication Lists']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
# cols = ['Pertinent Results']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
cols = ['Major Surgical Procedure']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The major procedures were:\n{context_data} \n"
cols = ['Discharge Diagnosis']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"

df = diagnosis_df
cols = ['icd_title']
cur_df = df[df['hadm_id'] == hadm_id]
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "


instruction = f"{topic} \n Instruction: {prompt}"
context_input = f"<text> Text:{context_str}</text> "

response = generate_with_model(instruction, context_input, model)
hpi_summary = response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
 Instruction: Using the following patient specific data,briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.Then, create a list of medical conditions and the associated treatments.

### Input:
<text> Text:The chief complaint was - fever 
The History of present illness: 
Mr ___ is a ___ with h/o stage IV Hodgkins c1d10 of ABVD, 
systemic sarcoidosis on high dose ster

In [38]:
# extract medication treatments, med comp use this
hadm_id = 22774359
topic = '''
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
from a patient's hospital admission. 
You will be provided with text and data from a specific patient's medical records and clinical notes
delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
Answer clearly and concisely.
'''.replace('\n', '')

# prompt = '''
# Given the the following medication list from admission and discharge, 
# summarize Which medications were new, discontinued, or changed dose and by what amount. 
# Associate each medication with one of the patient's major medical problems if possible. 
# Then create a table with columns "Name" ,"Admission dose", "Discharge dose" ,"Route", "Frequency" ,"Indication", 
# "New?", "Discontinued?", "Dose Change".
# '''.replace('\n', '')
prompt = '''
Given the the following medication list from admission and discharge, 
summarize Which medications were new, discontinued, or changed dose and by what amount. 
'''.replace('\n', '')

context_str = ''
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
cols = ['Medication Lists']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The admission medication lists:\n{context_data} \n"
# context_str = context_str + problem_list

instruction = f"{topic} \n Instruction: {prompt}"
context_input = f"<text> Text:{context_str}</text> "

response = generate_with_model(instruction, context_input, model)
Med_comp = response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
 Instruction: Given the the following medication list from admission and discharge, summarize Which medications were new, discontinued, or changed dose and by what amount. 

### Input:
<text> Text:The admission medication lists:
The Preadmission Medication list is accurate and complete.
1. Ondansetron 8 mg PO Q8H:PRN nausea 
2. PredniSONE 30 mg PO DAILY 
3. Prochlorperazine 10 mg PO Q6H:PRN nausea 
4. Fluconazole 400 mg PO Q24H 
5.

In [20]:
# extract Pertinent results related to problem list
hadm_id = 22774359
topic = '''
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
from a patient's hospital admission. 
You will be provided with text and data from a specific patient's medical records and clinical notes
delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
Answer clearly and concisely.
'''.replace('\n', '')

prompt = '''
What was the diagnostic workup and pertinent results during this patient's hospital admission?
Pertinent lab work includes positive or negative findings in blood lab or microbiology relating to the patient's conditions.
Pertinent radiology results include the type of study and a summary of the impressions.
Pertinent cardiology results include positive or negative findings from Echocardiogram or EKG relating to the patient's conditions.
Create list with the headings "Labs:", "Radiology:", "Cardiology", "Other".
'''.replace('\n', '')

context_str = problem_list
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
cols = ['Pertinent Results']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n').replace('=', '')
context_str = context_str + f"\n The pertinent results were:\n{context_data} \n"
cols = ['HPI']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The HPI: \n{context_data} \n"

instruction = f"{topic} \n Instruction: {prompt} \n"
context_input = f"<text> Text:{context_str}</text> "

response = generate_with_model(instruction, context_input, model)
pertinent_results = response


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
 Instruction: What was the diagnostic workup and pertinent results during this patient's hospital admission?Pertinent lab work includes positive or negative findings in blood lab or microbiology relating to the patient's conditions.Pertinent radiology results include the type of study and a summary of the impressions.Pertinent cardiology results include positive or negative findings from Echocardiogram or EKG relating to the patien

# generate BHC

In [22]:
hadm_id = 22774359
topic = '''
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
from a patient's hospital admission. 
You will be provided with text and data from a specific patient's medical records and clinical notes
delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
Answer clearly and concisely.
'''.replace('\n', '')

# Using the provided patient-specific data from the electronic healthcare record, generate a concise and brief summary of the patient's hospital course.  Focus on the key diagnosis, events, treatments, outcomes, and medication changes during the stay, including:

prompt = '''
Using the provided patient-specific data from the electronic healthcare record, generate a concise 
and brief summary of the patient's hospital course. Focus on the key diagnosis, major events, treatments, outcomes,
and medication changes during the stay. 
Start with a 1-3 sentence paragraph describing the patient’s major medical conditions, the reason for the 
hospital admission, and the initial treatments or diagnostic work up.  Then, for each medical condition in the provided list, describe the treatment course.
For each condition, describe the presenting symptom and severity, the diagnostic workup and results, 
treatments or procedures, medications, treatment response, or other events related to the condition. 
For chronic problems, indicate if there is a change to baseline medical management.
to this problem. For each problem, are there any pending results?  
  
'''.replace('\n', '')

context_str = ''
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
cols = ['CC']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = f"The chief complaint was - {context_data} \n"
cols = ['HPI']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The History of present illness: \n{context_data} \n"
cols = ['Medication Lists']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
# cols = ['Pertinent Results']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
cols = ['Discharge Diagnosis']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"
df = diagnosis_df
cols = ['icd_title']
cur_df = df[df['hadm_id'] == hadm_id]
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
context_str = context_str + problem_list + pertinent_results

# final_prompt = f"{topic} \n Instruction: {final_prompt}\n <text> Text:{context_str}</text>: "

# print("final prompt:\n", final_prompt, "\n******\n")

instruction = f"{topic} \n Instruction: {prompt}"
context_input = f"<text> Text:{context_str}</text> "

response = generate_with_model(instruction, context_input, model)
# BHC_intro = generate_summary_with_pipeline(final_prompt)
# print(BHC_intro)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
 Instruction: Using the provided patient-specific data from the electronic healthcare record, generate a concise and brief summary of the patient's hospital course. Focus on the key diagnosis, major events, treatments, outcomes,and medication changes during the stay. Start with a 1-3 sentence paragraph describing the patient’s major medical conditions, the reason for the hospital admission, and the initial treatments or diagnostic 

# generate discharge instructions

In [36]:

hadm_id = 22774359
topic = '''
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
from a patient's hospital admission. 
You will be provided with text and data from a specific patient's medical records and clinical notes
delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
Answer clearly and concisely.
'''.replace('\n', '')

prompt = '''
Given the following information from a patient's hospital stay,
Compose a letter to the patient that is courteous and easy to understand. 
There should be limited medical jargon and it should be written in layman's language.  
The letter will briefly summarize the reason for admission, the treatment course, relevant tests and results.  
It will also summarize anychanges to the patient's current medical management including medication changes. 
Is there any scheduled or recommended follow up?
If there was a surgery, what are the post-operative or wound care instructions?
Limit the letter to 3 paragraphs.
'''.replace('\n', '')

context_str = hpi_summary + Med_comp
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
# cols = ['CC']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = f"The chief complaint was - {context_data} \n"
# cols = ['HPI']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The History of present illness: \n{context_data} \n"
cols = ['Transitional Issues']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The ongoing issues to be addressed after discharge: \n{context_data} \n"
# cols = ['BHC']
# context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
# context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"


instruction = f"{topic} \n Instruction: {prompt}"
context_input = f"<text> Text:{context_str}</text> "

response = generate_with_model(instruction, context_input, model)
discharge_instructions = response


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
 Instruction: Given the following information from a patient's hospital stay,Compose a letter to the patient that is courteous and easy to understand. There should be limited medical jargon and it should be written in layman's language.  The letter will briefly summarize the reason for admission, the treatment course, relevant tests and results.  It will also summarize anychanges to the patient's current medical management includin